In [1]:
import numpy as np
import pandas as pd
import sklearn
import scipy

import sys
print(sys.version)

for i in [np, pd, sklearn, scipy]:
    print(i.__name__, i.__version__)

3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
numpy 1.23.5
pandas 1.5.3
sklearn 1.2.2
scipy 1.11.3


In [3]:
!wget https://raw.githubusercontent.com/sunkusun9/prods_asso/main/Dataset_07.csv

--2023-11-22 10:45:11--  https://raw.githubusercontent.com/sunkusun9/prods_asso/main/Dataset_07.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12890 (13K) [text/plain]
Saving to: ‘Dataset_07.csv’

Dataset_07.csv      100%[===================>]  12.59K  --.-KB/s    in 0s      

2023-11-22 10:45:11 (33.4 MB/s) - ‘Dataset_07.csv’ saved [12890/12890]



# 실습 문제 1

데이터셋: **Dataset_07.csv**, comma(",") 400rows, 9columns, UTF-8 인코딩

|컬럼|정의|Type|
|----|----|----|
|Serial_No|구분자 번호|Double|
|GRE| GRE 시험성적|Double|
|TOEFL| TOEFL 시험성적|Double|
|University_Rating| 대학 평가 그룹(1~5)|Double|
|SOP|자기 소개서 점수(1~5)|Double|
|LOR|추천서 점수(1~5)|Double|
|CGPA|학부 평량 평점(10점 만점 환산 점수)|Double|
|Research| 연구 참여 경험 여부(0/1)|Double|
|Chance_of Admit|합격 가능성|Double|

In [4]:
df_07 = pd.read_csv('Dataset_07.csv', index_col='Serial_No')
df_07.head()

,GRE,TOEFL,University_Rating,SOP,LOR,CGPA,Research,Chance_of_Admit
Serial_No,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,324,107,4,4.0,4.5,8.87,1,0.76
3,316,104,3,3.0,3.5,8.00,1,0.72
4,322,110,3,3.5,2.5,8.67,1,0.80
5,314,103,2,2.0,3.0,8.21,0,0.65


In [ ]:
df_07.shape

(400, 8)

In [ ]:
df_07.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400 entries, 1 to 400
Data columns (total 8 columns):
GRE                  400 non-null int64
TOEFL                400 non-null int64
University_Rating    400 non-null int64
SOP                  400 non-null float64
LOR                  400 non-null float64
CGPA                 400 non-null float64
Research             400 non-null int64
Chance_of_Admit      400 non-null float64
dtypes: float64(4), int64(4)
memory usage: 28.1 KB


**1**. 합격 가능성에 GRE, TOEFL, CGPA 점수 가운데 가장 영향이 큰 것이 어떤 점수인지 알아 보기 위해서 상관 분석을 수행한다.

- 피어슨(Pearson) 상관계수 값을 구한다.
- Chance_of_Admit와의 가장 큰 상관계수 값을 가지는 항목의 상관계수를 소수점 넷째 자리에서 반올림하여 셋째 자리까지 기술하시오.


(답안 예시) 0.123


In [ ]:
X = ['Chance_of_Admit', 'GRE', 'TOEFL', 'CGPA']
df_corr = df_07[X].corr()
df_corr

,Chance_of_Admit,GRE,TOEFL,CGPA
Chance_of_Admit,1.000000,0.802610,0.791594,0.873289
GRE,0.802610,1.000000,0.835977,0.833060
TOEFL,0.791594,0.835977,1.000000,0.828417
CGPA,0.873289,0.833060,0.828417,1.000000


In [ ]:
df_corr.loc['Chance_of_Admit', ['GRE', 'TOEFL', 'CGPA']]

GRE      0.802610
TOEFL    0.791594
CGPA     0.873289
Name: Chance_of_Admit, dtype: float64

In [ ]:
df_corr.loc['Chance_of_Admit', ['GRE', 'TOEFL', 'CGPA']].max()

0.8732890993553011

**0.873**

**2**. GRE 점수의 평균 이상을 받은 그룹과 평균 미만을 받은 그룹의 CGPA 평균은 차이가 있는지 검정을 하고자 한다.

- 적절한 검정 방법을 선택하고 양측 검정을 수행하시오 (등분산으로 가정)

- 검정 결과, 검정통계량의 추정치를 소수점 셋째 자리에서 반올림하여 소수점 두 자리까지 기술하시오.


Python 함수 가이드
```
scipy.stats에 있는 기능 활용
```

In [ ]:
from scipy.stats import ttest_ind

result = ttest_ind(
    df_07.loc[df_07['GRE'] >= df_07['GRE'].mean(), 'CGPA'],
    df_07.loc[df_07['GRE'] < df_07['GRE'].mean(), 'CGPA'],
    equal_var=True
)
result, result.statistic

(Ttest_indResult(statistic=19.443291692470982, pvalue=1.1127037052806161e-59),
 19.443291692470982)

**19.44**

**3**. Chance_of_Admit 확률이 0.5를 초과하면 합격으로, 이하이면 불합격으로 구분하고 로지스틱 회귀분석을 수행하시오.

- 원데이터만 사용하고, 원데이터 가운데 Serial_No와 Label은 모형에서 제외

- 각 설정값은 다음과 같이 지정하고, 언급되지 않은 사항은 기본 설정값을 사용하시오. Seed : 123

- 로지스틱 회귀분석 수행 결과에서 로지스틱 회귀계수의 절대값이 가장 큰 변수와 그 값을 기술하시오.

(로지스틱 회귀계수는 반올림하여 소수점 둘째 자리까지 / Intercept는 제외)

(답안 예시) abc, 0.12

Python 함수 가이드
```
sklearn.linear_model.LogisticRegression, solver='liblinear', random_state=123
```

In [ ]:
from sklearn.linear_model import LogisticRegression

clf_lr = LogisticRegression(solver='liblinear', random_state=123)
X_lr = ['GRE', 'TOEFL', 'University_Rating',  'SOP', 'LOR', 'CGPA', 'Research']
clf_lr.fit(df_07[X_lr], df_07['Chance_of_Admit'] > 0.5)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
np.abs(clf_lr.coef_[0])

array([0.08896188, 0.10548837, 0.05410951, 0.26490356, 1.01891822,
       2.06269373, 0.63495759])

**CGPA, 2.06**

In [5]:
!wget https://raw.githubusercontent.com/sunkusun9/prods_asso/main/Dataset_06.csv

--2023-11-22 10:45:43--  https://raw.githubusercontent.com/sunkusun9/prods_asso/main/Dataset_06.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 414271 (405K) [text/plain]
Saving to: ‘Dataset_06.csv’

Dataset_06.csv      100%[===================>] 404.56K  --.-KB/s    in 0.05s   

2023-11-22 10:45:43 (7.66 MB/s) - ‘Dataset_06.csv’ saved [414271/414271]



# 실습 문제 2

데이터셋: **Dataset_06.csv**, comma(",") 4323rows, 19columns, UTF-8 인코딩

|컬럼|정의|Type|
|--|---------|------|
|id|매물 번호|Double|
|date|날짜|String|
|price|거래 가격|Double|
|bedrooms|방 개수|Double|
|bathrooms|화장실 개수 (화장실은 있으나 샤워기 없는 경우 0.5로 처리)|Double|
|sqft_living|건축물 면적|Double|
|sqft_lot|대지 면적|Double|
|floors|건축물의 층수|Double|
|waterfront|강변 조망 가능 여부 (0 / 1)|Double|
|view|경관 (나쁨에서 좋음으로 0 ~ 4로 표시)|Double|
|condition|관리 상태 (나쁨에서 좋음으로 1 ~ 5로 표시)|Double|
|grade|등급 (낮음에서 높음으로 1 ~ 13으로 표시)|Double|
|sqft_above|지상 면적|Double|
|sqft_basement|지하실 면적|Double|
|yr_built|건축 연도|Double|
|yr_renovated|개축 연도|Double|
|zipcode|우편번호|Double|
|sqft_living15|15개의 인근 주택의 평균 건물 면적|Double|
|sqft_lot15	|15개의 인근 주택의 평균 대지 면적|Double|


In [6]:
df_06 = pd.read_csv('Dataset_06.csv')
df_06.head(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,sqft_living15,sqft_lot15
0,2597000006,20150309T000000,347500.0,3,1.50,1180,8353,1.0,0,0,3,7,1180,0,1960,0,98155,1710,8748
1,8813400345,20150414T000000,575000.0,2,1.00,980,3663,1.0,0,0,5,7,980,0,1909,0,98105,1620,3706
2,226059106,20150102T000000,489500.0,3,1.75,2090,65558,1.0,0,0,3,8,1330,760,1977,0,98072,2450,47178
3,8645500270,20141028T000000,246000.0,4,1.75,1720,7455,1.0,0,0,4,7,1020,700,1963,0,98058,1720,7700
4,7010700550,20141114T000000,595000.0,3,2.50,2030,5100,2.0,0,0,3,7,2030,0,2008,0,98199,1790,4380


In [36]:
df_06.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4323 entries, 0 to 4322
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             4323 non-null   int64  
 1   date           4323 non-null   object 
 2   price          4323 non-null   float64
 3   bedrooms       4323 non-null   int64  
 4   bathrooms      4323 non-null   float64
 5   sqft_living    4323 non-null   int64  
 6   sqft_lot       4323 non-null   int64  
 7   floors         4323 non-null   float64
 8   waterfront     4323 non-null   int64  
 9   view           4323 non-null   int64  
 10  condition      4323 non-null   int64  
 11  grade          4323 non-null   int64  
 12  sqft_above     4323 non-null   int64  
 13  sqft_basement  4323 non-null   int64  
 14  yr_built       4323 non-null   int64  
 15  yr_renovated   4323 non-null   int64  
 16  zipcode        4323 non-null   int64  
 17  sqft_living15  4323 non-null   int64  
 18  sqft_lot

**1** 강변 조망이 가능한지 여부(waterfront)에 따라 평균 주택 가격을 계산하고

조망이 가능한 경우와 그렇지 않은 경우의 평균 가격 차이의 절대값을 구하시오.

답은 소수점 이하는 버리고 정수부만 기술하시오.

(답안 예시) 1234567


In [8]:
a, b = df_06.groupby('waterfront')['price'].mean()
a, b, b - a

(532165.1307692308, 1699437.878787879, 1167272.748018648)

**1167272**

**2** price, bedrooms, bathrooms, sqft_living, sqft_lot, floors, yr_built 등 7개의 변수 간의 상관분석을 수행하고

price와의 상관계수의 절대값이 가장 큰 변수와 가장 작은 변수를 차례로 기술하시오.

(답안 예시) view, zipcode


In [9]:
X = ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'yr_built']
df_corr_06 = df_06[X].corr()
df_corr_06

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,yr_built
price,1.000000,0.279824,0.534277,0.709157,0.106341,0.270423,0.059895
bedrooms,0.279824,1.000000,0.468611,0.517502,0.017658,0.151279,0.137440
bathrooms,0.534277,0.468611,1.000000,0.751835,0.105676,0.495845,0.509019
sqft_living,0.709157,0.517502,0.751835,1.000000,0.180408,0.368670,0.323522
sqft_lot,0.106341,0.017658,0.105676,0.180408,1.000000,0.032762,0.062490
floors,0.270423,0.151279,0.495845,0.368670,0.032762,1.000000,0.505070
yr_built,0.059895,0.137440,0.509019,0.323522,0.062490,0.505070,1.000000


In [10]:
mn, mx = df_corr_06.loc['price', 'bedrooms': ].abs().sort_values().iloc[[0, -1]]
mx, mn,  mx -mn

(0.7091565835706303, 0.059895410779646746, 0.6492611727909835)

**3** id, date, 그리고 zipcode를 제외한 모든 변수를 독립변수로, price를 종속변수로 하여 회귀분석을 수행하시오.

통계적 유의성을 갖지 못하는 독립변수를 제거하면 회귀 모형에 남는 변수는 모두 몇 개인가?  (유의 수준: 0.05)

이 때 음의 회귀계수를 가지는 변수는 몇 개인가?


Python 함수 가이드
```
from sklearn.linear_model import LinearRegression 또는
from statsmodels.formula.api import ols 중 적합한 것을 사용하세요
```

In [11]:
from statsmodels.formula.api import ols

In [12]:
X = [i for i in df_06.columns if i not in ['id', 'date', 'zipcode', 'price']]
formula = 'price ~ ' + '+'.join(X)
formula

'price ~ bedrooms+bathrooms+sqft_living+sqft_lot+floors+waterfront+view+condition+grade+sqft_above+sqft_basement+yr_built+yr_renovated+sqft_living15+sqft_lot15'

In [13]:
lm_06 = ols(formula, data=df_06).fit()
lm_06.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.681
Model:                            OLS   Adj. R-squared:                  0.680
Method:                 Least Squares   F-statistic:                     656.0
Date:                Wed, 22 Nov 2023   Prob (F-statistic):               0.00
Time:                        10:45:50   Log-Likelihood:                -58960.
No. Observations:                4323   AIC:                         1.180e+05
Df Residuals:                    4308   BIC:                         1.180e+05
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      6.302e+06   2.95e+05     21.401      0.000    5.73e+06    6.88e+06
bedrooms        -2.5e+04   3680.510     -6.791      0.000   -3.22e+04   -1.78e+04
bathrooms      5.179e+04   7163.408      7.230      0.000    3.77e+04    6.58e+04
sqft_living      92.3080      5.074     18.192      0.000      82.360     102.256
sqft_lot          0.0054      0.093      0.059      0.953      -0.176       0.187
floors         3.748e+04   8141.392      4.604      0.000    2.15e+04    5.34e+04
waterfront     6.277e+05   3.91e+04     16.064      0.000    5.51e+05    7.04e+05
view            4.04e+04   4797.344      8.421      0.000     3.1e+04    4.98e+04
condition      1.571e+04   5317.387      2.954      0.003    5280.918    2.61e+04
grade          1.306e+05   4700.604     27.789      0.000    1.21e+05     1.4e+05
sqft_above       33.7358      4.981      6.773      0.000      23.971      43.501
sqft_basement    58.5723      5.840     10.030      0.000      47.123      70.021
yr_built      -3666.8957    150.805    -24.316      0.000   -3962.551   -3371.241
yr_renovated     17.7032      8.265      2.142      0.032       1.499      33.908
sqft_living15    23.0114      7.561      3.044      0.002       8.188      37.834
sqft_lot15       -0.2323      0.142     -1.637      0.102      -0.511       0.046
==============================================================================
Omnibus:                     2170.721   Durbin-Watson:                   2.011
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            38362.875
Skew:                           1.971   Prob(JB):                         0.00
Kurtosis:                      17.051   Cond. No.                     1.94e+17
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 4.29e-22. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

**sqft_lot, sqft_lot15**

In [14]:
len([i for i in df_06.columns if i not in ['id', 'date', 'zipcode', 'price']]) - 2

13

**bedrooms, yr_built**

**13, 2**